In [2]:
import sys, os
sys.path.append('../misc')
from helpers import *
import pandas as pd

In [4]:
#df = get_full_table('./input/training_variants','./input/training_text')
df = pd.read_csv('../data/test_filtered')

In [5]:
def var_in_text(text):
    import re
    text = text.lower()
    var_list = []
    for var in re.finditer('[a-z]+[0-9]+[a-z]*', text):
        var_list.append(var.group(0))
    return set(var_list)

def get_gene_list(gene):
    import json
    with open('../feature_engineer/gene_query_result.json','r') as f:
        data = json.load(f)
    data2 = data[gene]['response']['docs'][0]
    l = []
    if 'symbol' in data2:
        l.append(data2['symbol'])
    if 'prev_symbol' in data2:
        l = l+data2['prev_symbol']
    if 'alias_symbol' in data2:
        l = l+data2['alias_symbol']
    return [i.lower() for i in l]

def get_var_list(var):
    amino_dict = {'a': 'ala', 'c': 'cys', 'd': 'asp', 'e': 'glu', 'f': 'phe', 'g': 'gly', 'h': 'his', 'i': 'ile',\
     'k': 'lys', 'l': 'leu', 'm': 'met', 'n': 'asn', 'p': 'pro', 'q': 'gln', 'r': 'arg', 's': 'ser', \
     't': 'thr', 'v': 'val', 'w': 'trp', 'y': 'tyr', 'b':'asx', 'z': 'glx'}
    import re
    var = var.lower()
    
    m = re.match('([a-z])([0-9]+)([a-z])', var)
    if m:
        return [var, m.group(1)+str(int(m.group(2))+3)+m.group(3),\
                    m.group(1)+str(int(m.group(2))-3)+m.group(3),\
                amino_dict[m.group(1)]+m.group(2)+amino_dict[m.group(3)],\
                amino_dict[m.group(1)]+str(int(m.group(2))+3)+amino_dict[m.group(3)],\
               amino_dict[m.group(1)]+str(int(m.group(2))-3)+amino_dict[m.group(3)]],\
                [m.group(1)+'[\d]+'+m.group(3), \
                 amino_dict[m.group(1)]+'[\d]+'+amino_dict[m.group(3)],\
                 m.group(1)+m.group(2),\
                 amino_dict[m.group(1)]+m.group(2)]
    
     
    m = re.match('([a-z])([0-9]+)([\*])', var)
    if m:
        return [var, m.group(1)+str(int(m.group(2))+3)+m.group(3),\
                    m.group(1)+str(int(m.group(2))-3)+m.group(3),\
                amino_dict[m.group(1)]+m.group(2)+m.group(3),\
                amino_dict[m.group(1)]+str(int(m.group(2))+3)+m.group(3),\
               amino_dict[m.group(1)]+str(int(m.group(2))-3)+m.group(3)],\
                [m.group(1)+'[\d]+'+'\*', \
                 amino_dict[m.group(1)]+'[\d]+'+'\*',\
                 m.group(1)+m.group(2),\
                 amino_dict[m.group(1)]+m.group(2)]
        
    if re.search('trunc', var):
        return [var, 'truncat'],[]
        
    elif re.search('del', var):
        return [var, 'delet'],[]
        
    elif re.search('amp', var):
        return [var, 'amplific'],[]
        
    elif re.search('dup', var):
        return [var, 'duplicat'],[]
        
    elif re.search('ins', var):
        return [var, 'insert'],[]
     
    elif re.search('fus', var):
        return [var, 'fusion'],[]
    
    elif re.search('splice', var):
        return [var, 'splice'],[]
        
    elif re.search('over', var):
        return [var, 'overexpress'],[]
    
    elif re.search('exon', var):
        return [var, 'exon'],[]
        
    elif re.search('promo', var):
        return [var, 'promo'],[]
    
    elif re.search('egfr', var):
        return [var, 'egfr'],[]
    
    else:
        return [var],[]
    

In [6]:
def get_partial_text(gene, var, text):
    import re
    l = get_var_list(var)[0]
    l2 = get_var_list(var)[1]
    l3 = get_gene_list(gene)
    text = text.lower()
    start = []
    #print(l, l2)
    for i in l:
        p = re.compile(i)
        for m in p.finditer(text):
            start.append(m.start())
    if len(start)==0 and len(l2)>0:
        for i in l2:
            p = re.compile(i)
            for m in p.finditer(text):
                start.append(m.start())
    if len(start)==0:
        for i in l3:
            p = re.compile(i)
            for m in p.finditer(text):
                start.append(m.start())
                
    start.sort()
    return start
        

In [7]:
import pandas as pd
passage = 200
df['SeparateText'] = pd.Series(['']*df.shape[0])
for i in range(df.shape[0]):
    s = get_partial_text(df['Gene'].iloc[i], df['Variation'].iloc[i], df['Text'].iloc[i])
    text = df['Text'].iloc[i]
    if len(s)==0:
        df['SeparateText'].iloc[i] += text
    else:
        text_length = len(text)
        for start in s:
            try:
                df['SeparateText'].iloc[i] += \
                ' ' + text[max(0,start-passage):min(start+passage, text_length)].split(' ', 1)[1].rsplit(' ', 1)[0] 
            except:
                print('this part goes wrong: {}.{}: s = {}'.format(i, start, len(s)))
    if len(df['SeparateText'].iloc[i]) == 0:
        df['SeparateText'].iloc[i] += text
        
    if i%100 == 0:
        print(i)

//anaconda/envs/cancer/lib/python3.6/site-packages/pandas/core/indexing.py:179: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


0
this part goes wrong: 47.35051: s = 1
this part goes wrong: 54.34587: s = 1
this part goes wrong: 85.35797: s = 1
100
200
this part goes wrong: 241.35015: s = 1
this part goes wrong: 246.34217: s = 1
300


In [94]:
#for i in failed:
#    passage = 200
#    print(get_gene_list(df['Gene'].iloc[i]))
#    print(get_var_list(df['Variation'].iloc[i]))
#    s = get_partial_text(df['Gene'].iloc[i], df['Variation'].iloc[i], df['Text'].iloc[i])
#    text = df['Text'].iloc[i]
#    text_length = len(text)
#    print(s)
#    print('------{}-------'.format(i))
#    for start in s:
#        print(start)
#        print(text[max(0,start-passage):min(start+passage, text_length)])
#        print('--------')

In [95]:
#i = 23
#print(get_gene_list(df['Gene'].iloc[i]))
#print(get_var_list(df['Variation'].iloc[i]))
#s = get_partial_text(df['Gene'].iloc[i], df['Variation'].iloc[i], df['Text'].iloc[i])
#text = df['Text'].iloc[i]
#for start in s:
#    print(start)
#    text_length = len(text)
#    print(text[max(0,start-passage):min(start+passage, text_length)].split(' ', 1)[1].rsplit(' ', 1)[0])
#    print('--------')

In [96]:
#i = 2774
#print(get_gene_list(df['Gene'].iloc[i]))
#print(get_var_list(df['Variation'].iloc[i]))
#print(var_in_text(df['Text'].iloc[i]))
#text = df['Text'].iloc[i]
#print(text[max(0,start-passage):min(start+passage, text_length)].split(' ', 1))

In [97]:
#for i in df['Text'].unique():
#    c = pd.Series(list(df[df['Text']==i]['ID'])).unique()
#    if len(c)>1:
#        print(df[df['Text']==i])

In [8]:
df.to_csv('../data/test_filtered_SeparateText.csv',\
                  index=False)

In [11]:
df.iloc[100]

ID                                                           3421
Gene                                                         ELF3
Variation                                                   A312P
Class                                                           1
Text            The ETS transcription factor family is compris...
SeparateText     with 15 μg of the mTβR-II -108/+56(B/A) P/R c...
Name: 100, dtype: object

In [12]:
print(df.iloc[100]['SeparateText'])

 with 15 μg of the mTβR-II -108/+56(B/A) P/R construct. Where indicated, the cells were also transfected with 0.5 or 1 μg of an N-terminal GFP fusion expression vector for Elf3, Elf3I279P, Elf3A312P, Elf3K320E, or Elf3R331P. Activities were assayed and normalized as described under “Experimental Procedures.” The promoter activity of each construct is calculated relative to the CAT activity activity of the mTβR-II -108/+56(B/A) P/R construct increased 4–6-fold over the basal activity of the P/R gene construct (Fig. 1C). As predicted, all three helix mutations (GFP-Elf3I279P, GFP-Elf3A312P, and GFP-Elf3R331P) resulted in a complete loss of Elf3 stimulation of mTβR-II -108/+56(B/A). In contrast, other non-helix breaking mutations within the turn between helix 2 and 3 (K320E) and independent of the possible influence of an AID. EMSA using equal amounts of Elf3ΔN270 with or without the ETS domain mutations detected a loss of binding with mutants Elf3ΔN270I279P, Elf3ΔN270A312P, and Elf3ΔN270